In [ ]:
import os, matplotlib.pyplot as plt, seaborn as sns

BASE_DIR = os.getcwd()
TRAIN_DIR = os.path.abspath("../Teeth DataSet/Teeth_Dataset/Training")
os.listdir(TRAIN_DIR)

In [ ]:
class_counts = {}
for class_name in os.listdir(TRAIN_DIR):
    class_path = os.path.join(TRAIN_DIR, class_name)
    class_counts[class_name] = len(os.listdir(class_path))

print(class_counts)

In [ ]:
sns.barplot(x=class_counts.keys(), y=class_counts.values())

In [ ]:
import matplotlib.image as mpimg

fig, axes = plt.subplots(1,7,figsize=(20,10))
classes = os.listdir(TRAIN_DIR)

for i, class_name in enumerate(classes):
    class_folder= os.path.join(TRAIN_DIR, class_name)
    img_name = os.listdir(class_folder)[0]
    img_path = os.path.join(class_folder, img_name)

    img = mpimg.imread(img_path)
    axes[i].imshow(img)

    axes[i].set_title(class_name)
    axes[i].axis('off')

plt.show()